In [8]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely.geometry as sgeo

In [9]:
if "snakemake" in locals():
    input_passengers_path = snakemake.input["passengers"]
    input_airports_path = snakemake.input["airports"]
    output_path = snakemake.output[0]
    seed = snakemake.params["seed"]
    
else:
    input_passengers_path = "../../../results/brussels/population/passengers_with_postcode_seed1000.parquet"
    input_airports_path = "../../../resources/brussels/airport/location.gpkg"
    output_path = "../../../results/brussels/trips/passenger_trips.parquet"
    seed = 0

In [10]:
# Load data
df_passengers = gpd.read_parquet(input_passengers_path)
df_airports = gpd.read_file(input_airports_path)

In [11]:
# Prepare random state
random = np.random.RandomState(seed)

In [12]:
# Sample access/egress
df_passengers["is_access"] = random.random_sample(len(df_passengers)) < 0.5

In [13]:
# Sample destinations
destinations = df_airports.sample(
    n = len(df_passengers), replace = True, weights = "weight", random_state = random)["geometry"]

# Construct geometry
df_passengers["geometry"] = [
    sgeo.LineString([first, second]) if is_access else sgeo.LineString([second, first])
    for first, second, is_access in zip(
        df_passengers["geometry"], destinations, df_passengers["is_access"])
]

In [14]:
# Sample reference times
df_passengers["reference_time"] = df_passengers["departure_hour"].values * 3600
df_passengers["reference_time"] += random.random_sample(len(df_passengers)) * 3600

In [15]:
if output_path.endswith("parquet"):
    df_passengers.to_parquet(output_path)
else:
    df_passengers.to_file(output_path)